In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from scipy.optimize import curve_fit
import os

In [9]:
datapath = '/home/lautaro/Workspace/ib/code/codedoc/realdata/redes/FAO'
datapath_win = 'D:\\Workspace\\code\\realdata\\redes\\FAO'
# Comento esta linea si estoy en linux
datapath = datapath_win
os.listdir(datapath)

['coordinate-countries-fixed.csv',
 'edges.csv',
 'exports_by_country.csv',
 'exports_by_country_soja.csv',
 'FAO_manlio',
 'gprops copy.csv',
 'gprops-fixed.csv',
 'gprops.csv',
 'imports_by_country.csv',
 'imports_by_country_soja.csv',
 'net_trade_by_country.csv',
 'net_trade_by_country_soja.csv',
 'nodes-fixed.csv',
 'nodes-merged.csv',
 'nodes.csv',
 'not_nodes.csv',
 'ordered_nodes.csv',
 'total_trade_by_country.csv']

In [10]:
products = eval(open(os.path.join(datapath, "gprops-fixed.csv")).read())

def get_all_edges():
    all_edges = pd.read_csv(os.path.join(datapath, "edges.csv"))
    all_edges = all_edges.rename(
        columns={
            "# source": "source",
            " target": "target",
            " weight": "weight",
            " layer": "layer",
        }
    )
    return all_edges

def get_edges_by_layer(layer=0):
    all_edges = get_all_edges()

    unique_count = all_edges[["source", "target"]].values.flatten()
    unique_count = len(set(unique_count))

    one_layer_edges = all_edges[all_edges["layer"] == layer + 1]

    del all_edges
    return one_layer_edges[
        one_layer_edges["source"] != one_layer_edges["target"]
    ]

In [11]:
full_dataset = get_all_edges()
# Descomento si me quiero sacar de encima las subdivisiones de China
# Drop all entries of full_dataset where either source or target are equal to 7, 8, 9 or 139 (Hong Kong, China mainland, Taiwan, Macao)
# full_dataset = full_dataset[~full_dataset["source"].isin([7, 8, 9, 139])]
# full_dataset = full_dataset[~full_dataset["target"].isin([7, 8, 9, 139])]

In [20]:
# Para agregarle los nombres a la tabla
nodes = pd.read_csv(os.path.join(datapath, 'nodes-fixed.csv'))
nodes = nodes.rename(columns={'# index': 'index', ' name': 'name'})
nodes = nodes.drop(columns=[' nodeLabel', ' _pos'])
nodes['name'] = nodes['name'].str.replace('_', ' ')

In [21]:
# Agrupo por pais en source y target y sumo los pesos
export_sum = full_dataset.groupby('source')['weight'].sum().reset_index()
export_sum['weight'] = export_sum['weight'] / 1000
export_sum = export_sum.rename(columns={'source': 'index', 'weight': 'exports (m usd)'})
export_sum = export_sum.merge(nodes, on='index', how='inner')
export_sum = export_sum[['index', 'name', 'exports (m usd)']]
# export_sum.to_csv(os.path.join(datapath, 'exports_by_country.csv'), index=False)

import_sum = full_dataset.groupby('target')['weight'].sum().reset_index()
import_sum['weight'] = import_sum['weight'] / 1000
import_sum = import_sum.rename(columns={'target': 'index', 'weight': 'imports (m usd)'})
import_sum = import_sum.merge(nodes, on='index', how='inner')
import_sum = import_sum[['index', 'name', 'imports (m usd)']]
# import_sum.to_csv(os.path.join(datapath, 'imports_by_country.csv'), index=False)

In [22]:
# Outer join de export e import
merged_df = export_sum.merge(import_sum, left_on='index', right_on='index', how='outer')
merged_df['exports (m usd)'].fillna(0, inplace=True)
merged_df['imports (m usd)'].fillna(0, inplace=True)
merged_df = merged_df.rename(columns={'name_x': 'name'})
merged_df

,index,name,exports (m usd),name_y,imports (m usd)
0,1,Australia,2.569,Australia,0.944
1,2,Austria,22.724,Austria,46.912
2,3,Belgium,74.625,Belgium,265.198
3,4,Brazil,12670.473,Brazil,44.792
4,5,Canada,1212.944,Canada,104.508
...,...,...,...,...,...
125,177,NaN,0.000,Saint Vincent and the Grenadines,0.574
126,181,NaN,0.000,French Polynesia,0.022
127,186,NaN,0.000,Senegal,0.006
128,189,NaN,0.000,Bhutan,0.017


In [15]:
# Calculate the subtraction of 'value' from source minus 'value' from target
merged_df['net_trade (m usd)'] = merged_df['exports (m usd)'] - merged_df['imports (m usd)']
merged_df['i + e (m usd)'] = merged_df['exports (m usd)'] + merged_df['imports (m usd)']

# Select only the relevant columns
result_df = merged_df[['index', 'name', 'net_trade (m usd)', 'i + e (m usd)']]
# result_df.to_csv(os.path.join(datapath, 'net_trade_by_country.csv'), index=False)

result_df

,index,name,net_trade (m usd),i + e (m usd)
0,0,Afghanistan,-459.129,921.263
1,1,Australia,20130.750,38940.476
2,2,Austria,-2100.364,21939.596
3,3,Belgium,-1237.316,62768.580
4,4,Brazil,59123.039,78147.503
...,...,...,...,...
209,209,Cayman Islands,3.046,3.046
210,210,Mayotte,0.429,0.429
211,211,Maldives,-229.878,232.186
212,212,Mauritania,-350.004,357.896


### 24/10: Quiero hacer esta misma tabla neta solo para la soja

In [16]:
full_dataset = get_edges_by_layer(layer=344)
# Drop all entries of full_dataset where either source or target are equal to 7, 8, 9 or 139 (Hong Kong, China mainland, Taiwan, Macao)
# Also drop entries where source or target are equal to 92 (Unspecified)
full_dataset = full_dataset[~full_dataset["source"].isin([7, 8, 9, 139, 92])]
full_dataset = full_dataset[~full_dataset["target"].isin([7, 8, 9, 139, 92])]
full_dataset

# Para agregarle los nombres a la tabla
nodes = pd.read_csv(os.path.join(datapath, 'nodes-fixed.csv'))
nodes = nodes.rename(columns={'# index': 'index', ' name': 'name'})
nodes = nodes.drop(columns=[' nodeLabel', ' _pos'])
nodes['name'] = nodes['name'].str.replace('_', ' ')

# Agrupo por pais en source y target y sumo los pesos
export_sum = full_dataset.groupby('source')['weight'].sum().reset_index()
export_sum['weight'] = export_sum['weight'] / 1000
export_sum = export_sum.rename(columns={'source': 'index', 'weight': 'exports (m usd)'})
export_sum = export_sum.merge(nodes, on='index', how='inner')
export_sum = export_sum[['index', 'name', 'exports (m usd)']]
# export_sum.to_csv(os.path.join(datapath, 'exports_by_country_soja.csv'), index=False)

import_sum = full_dataset.groupby('target')['weight'].sum().reset_index()
import_sum['weight'] = import_sum['weight'] / 1000
import_sum = import_sum.rename(columns={'target': 'index', 'weight': 'imports (m usd)'})
import_sum = import_sum.merge(nodes, on='index', how='inner')
import_sum = import_sum[['index', 'name', 'imports (m usd)']]
# import_sum.to_csv(os.path.join(datapath, 'imports_by_country_soja.csv'), index=False)

# Outer join de export e import
merged_df = export_sum.merge(import_sum, left_on='index', right_on='index', how='outer')
merged_df['exports (m usd)'].fillna(0, inplace=True)
merged_df['imports (m usd)'].fillna(0, inplace=True)
merged_df = merged_df.rename(columns={'name_x': 'name'})
merged_df['name'].fillna(merged_df['name_y'], inplace=True)
merged_df



,index,name,exports (m usd),name_y,imports (m usd)
0,1,Australia,2.569,Australia,0.944
1,2,Austria,22.724,Austria,46.912
2,3,Belgium,74.625,Belgium,265.198
3,4,Brazil,12670.473,Brazil,44.792
4,5,Canada,1212.944,Canada,104.508
...,...,...,...,...,...
125,177,Saint Vincent and the Grenadines,0.000,Saint Vincent and the Grenadines,0.574
126,181,French Polynesia,0.000,French Polynesia,0.022
127,186,Senegal,0.000,Senegal,0.006
128,189,Bhutan,0.000,Bhutan,0.017


In [18]:
# Calculate the subtraction of 'value' from source minus 'value' from target
merged_df['net_trade (m usd)'] = merged_df['exports (m usd)'] - merged_df['imports (m usd)']

# Select only the relevant columns
result_df = merged_df[['index', 'name', 'net_trade (m usd)']]
# result_df.to_csv(os.path.join(datapath, 'net_trade_by_country_soja.csv'), index=False)

result_df

,index,name,net_trade (m usd)
0,1,Australia,1.625
1,2,Austria,-24.188
2,3,Belgium,-190.573
3,4,Brazil,12625.681
4,5,Canada,1108.436
...,...,...,...
125,177,Saint Vincent and the Grenadines,-0.574
126,181,French Polynesia,-0.022
127,186,Senegal,-0.006
128,189,Bhutan,-0.017
